In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import load_model
from sklearn.ensemble import VotingClassifier
import pickle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import time

In [2]:
IoT = pd.read_csv('../../data/test_heterogeneous.csv')
IoT = IoT.iloc[0:5000]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['fridge_temp_condition']=encoder.fit_transform(IoT['fridge_temp_condition'])
IoT['door_state']=encoder.fit_transform(IoT['door_state'])
IoT['door_sphone_signal']=encoder.fit_transform(IoT['door_sphone_signal'])
IoT['gps_type']=encoder.fit_transform(IoT['gps_type'])
IoT['ml_light_status']=encoder.fit_transform(IoT['ml_light_status'])

IoT.head()

,date-time,fridge_temperature,fridge_temp_condition,door_state,door_sphone_signal,gps_latitude,gps_longitude,gps_type,modbus_FC1_Read_Input_Register,modbus_FC2_Read_Discrete_Value,modbus_FC3_Read_Holding_Register,modbus_FC4_Read_Coil,ml_motion_status,ml_light_status,thermostat_current_temperature,thermostat_status,weather_temperature,weather_pressure,weather_humidity
0,31-03-19~12:36:52,13.10,0,0,0,0.0,10.0,2,11287.0,3537.0,27844.0,30527.0,1.0,1,25.0,0.0,31.788508,1.035,32.036579
1,31-03-19~12:36:53,8.65,0,0,0,0.0,10.0,2,11287.0,3537.0,27844.0,30527.0,0.0,0,25.0,0.0,41.630997,1.035,30.886165
2,31-03-19~12:36:54,2.00,1,0,0,0.0,10.0,2,11287.0,3537.0,27844.0,30527.0,0.0,0,25.0,0.0,42.256959,1.035,19.755908
3,31-03-19~12:36:55,4.80,1,0,0,0.0,10.0,2,53287.0,1463.0,33518.0,23014.0,1.0,1,25.0,0.0,49.116581,1.035,78.949621
4,31-03-19~12:36:56,10.70,0,0,0,0.0,10.0,2,53287.0,1463.0,33518.0,23014.0,1.0,1,25.0,0.0,24.017085,1.035,40.001059


In [3]:
x = IoT.drop(['gps_type', 'date-time'], axis=1)
y = IoT['gps_type']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model3 = pickle.load(open('../h5s/random-forest.h5', 'rb'))
model4 = pickle.load(open('../h5s/logistic-regression.h5', 'rb'))
model5 = pickle.load(open('../h5s/linear-discriminant-analysis.h5', 'rb'))
model6 = pickle.load(open('../h5s/kNN.h5', 'rb'))
model7 = pickle.load(open('../h5s/CART.h5', 'rb'))
model8 = pickle.load(open('../h5s/support-vector-machine.h5', 'rb'))
model9 = pickle.load(open('../h5s/naive-bayes.h5', 'rb'))

In [5]:
voting = VotingClassifier(estimators=[('RF', model3), ('LR', model4), ('LDA', model5), ('KNN', model6), ('CART', model7), ('SVM', model8), ('Naive-Bayes', model9)], voting='hard')
voting.fit(x_train, y_train) 

for clf in (model3, model4, model5, model6, model7, model8, model9, voting):
    train_start_time=time.time()
    clf.fit(x_train, y_train)
    train_end_time=time.time()
    test_start_time=time.time()
    y_pred = clf.predict(x_test)
    test_end_time=time.time()
    print(clf.__class__.__name__, 'accuracy : ', accuracy_score(y_test, y_pred))
    print(clf.__class__.__name__, 'precision : ', precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'recall : ', recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'f1_score : ', f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'train time : ', train_end_time-train_start_time, ' sec')
    print(clf.__class__.__name__, 'test time : ', test_end_time-test_start_time, ' sec')
    print('\n')

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForestClassifier accuracy :  0.977
RandomForestClassifier precision :  0.977449485049834
RandomForestClassifier recall :  0.977
RandomForestClassifier f1_score :  0.9768762068251823
RandomForestClassifier train time :  0.7020723819732666  sec
RandomForestClassifier test time :  0.024388790130615234  sec




c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression accuracy :  0.593
LogisticRegression precision :  0.5815575176655041
LogisticRegression recall :  0.6670416197975253
LogisticRegression f1_score :  0.6178249914237657
LogisticRegression train time :  0.2342512607574463  sec
LogisticRegression test time :  0.001039266586303711  sec


LinearDiscriminantAnalysis accuracy :  0.815
LinearDiscriminantAnalysis precision :  0.8126742810220616
LinearDiscriminantAnalysis recall :  0.815
LinearDiscriminantAnalysis f1_score :  0.8111849403616522
LinearDiscriminantAnalysis train time :  0.014964818954467773  sec
LinearDiscriminantAnalysis test time :  0.000997781753540039  sec


KNeighborsClassifier accuracy :  0.531
KNeighborsClassifier precision :  0.5299997339954606
KNeighborsClassifier recall :  0.531
KNeighborsClassifier f1_score :  0.5051216209427768
KNeighborsClassifier train time :  0.0019948482513427734  sec
KNeighborsClassifier test time :  0.11081361770629883  sec


DecisionTreeClassifier accuracy :  0.965
DecisionTree

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier accuracy :  0.867
VotingClassifier precision :  0.885160834977902
VotingClassifier recall :  0.867
VotingClassifier f1_score :  0.865847235864722
VotingClassifier train time :  2.069828748703003  sec
VotingClassifier test time :  0.46225690841674805  sec




In [6]:
voting = VotingClassifier(estimators=[('SVM', model8), ('Naive-Bayes', model9), ('RF', model3), ('LR', model4), ('LDA', model5), ('KNN', model6), ('CART', model7)], voting='hard')
voting.fit(x_train, y_train) 

for clf in (model8, model9, model3, model4, model5, model6, model7, voting):
    train_start_time=time.time()
    clf.fit(x_train, y_train)
    train_end_time=time.time()
    test_start_time=time.time()
    y_pred = clf.predict(x_test)
    test_end_time=time.time()
    print(clf.__class__.__name__, 'accuracy : ', accuracy_score(y_test, y_pred))
    print(clf.__class__.__name__, 'precision : ', precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'recall : ', recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'f1_score : ', f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'train time : ', train_end_time-train_start_time, ' sec')
    print(clf.__class__.__name__, 'test time : ', test_end_time-test_start_time, ' sec')
    print('\n')

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVC accuracy :  0.541
SVC precision :  0.6618650750622475
SVC recall :  0.689171974522293
SVC f1_score :  0.5950610760078359
SVC train time :  1.1749069690704346  sec
SVC test time :  0.3313019275665283  sec


GaussianNB accuracy :  0.702
GaussianNB precision :  0.7570147173319124
GaussianNB recall :  0.702
GaussianNB f1_score :  0.6960336385884475
GaussianNB train time :  0.0029501914978027344  sec
GaussianNB test time :  0.0009968280792236328  sec


RandomForestClassifier accuracy :  0.978
RandomForestClassifier precision :  0.9783890744903496
RandomForestClassifier recall :  0.978
RandomForestClassifier f1_score :  0.977876593701376
RandomForestClassifier train time :  0.5861313343048096  sec
RandomForestClassifier test time :  0.020943164825439453  sec




c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression accuracy :  0.593
LogisticRegression precision :  0.5815575176655041
LogisticRegression recall :  0.6670416197975253
LogisticRegression f1_score :  0.6178249914237657
LogisticRegression train time :  0.22613000869750977  sec
LogisticRegression test time :  0.0019919872283935547  sec


LinearDiscriminantAnalysis accuracy :  0.815
LinearDiscriminantAnalysis precision :  0.8126742810220616
LinearDiscriminantAnalysis recall :  0.815
LinearDiscriminantAnalysis f1_score :  0.8111849403616522
LinearDiscriminantAnalysis train time :  0.013962030410766602  sec
LinearDiscriminantAnalysis test time :  0.0019996166229248047  sec


KNeighborsClassifier accuracy :  0.531
KNeighborsClassifier precision :  0.5299997339954606
KNeighborsClassifier recall :  0.531
KNeighborsClassifier f1_score :  0.5051216209427768
KNeighborsClassifier train time :  0.0020372867584228516  sec
KNeighborsClassifier test time :  0.10351061820983887  sec


DecisionTreeClassifier accuracy :  0.967
DecisionT

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier accuracy :  0.866
VotingClassifier precision :  0.8839355657563112
VotingClassifier recall :  0.866
VotingClassifier f1_score :  0.8648017524713283
VotingClassifier train time :  2.0852701663970947  sec
VotingClassifier test time :  0.4848606586456299  sec




In [8]:
voting = VotingClassifier(estimators=[('CART', model7), ('LR', model4), ('SVM', model8), ('Naive-Bayes', model9), ('RF', model3), ('LDA', model5), ('KNN', model6)], voting='hard')
voting.fit(x_train, y_train) 

for clf in (model7, model4, model8, model9, model3, model5, model6, voting):
    train_start_time=time.time()
    clf.fit(x_train, y_train)
    train_end_time=time.time()
    test_start_time=time.time()
    y_pred = clf.predict(x_test)
    test_end_time=time.time()
    print(clf.__class__.__name__, 'accuracy : ', accuracy_score(y_test, y_pred))
    print(clf.__class__.__name__, 'precision : ', precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'recall : ', recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'f1_score : ', f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'train time : ', train_end_time-train_start_time, ' sec')
    print(clf.__class__.__name__, 'test time : ', test_end_time-test_start_time, ' sec')
    print('\n')

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


DecisionTreeClassifier accuracy :  0.967
DecisionTreeClassifier precision :  0.9669252694432902
DecisionTreeClassifier recall :  0.967
DecisionTreeClassifier f1_score :  0.9669017545188257
DecisionTreeClassifier train time :  0.029920101165771484  sec
DecisionTreeClassifier test time :  0.000997304916381836  sec




c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression accuracy :  0.593
LogisticRegression precision :  0.5815575176655041
LogisticRegression recall :  0.6670416197975253
LogisticRegression f1_score :  0.6178249914237657
LogisticRegression train time :  0.21954584121704102  sec
LogisticRegression test time :  0.0010495185852050781  sec


SVC accuracy :  0.541
SVC precision :  0.6618650750622475
SVC recall :  0.689171974522293
SVC f1_score :  0.5950610760078359
SVC train time :  1.2746410369873047  sec
SVC test time :  0.8107390403747559  sec


GaussianNB accuracy :  0.702
GaussianNB precision :  0.7570147173319124
GaussianNB recall :  0.702
GaussianNB f1_score :  0.6960336385884475
GaussianNB train time :  0.008977174758911133  sec
GaussianNB test time :  0.002990245819091797  sec


RandomForestClassifier accuracy :  0.979
RandomForestClassifier precision :  0.9793795506808258
RandomForestClassifier recall :  0.979
RandomForestClassifier f1_score :  0.9789517414638303
RandomForestClassifier train time :  0.9444766044616

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier accuracy :  0.867
VotingClassifier precision :  0.885160834977902
VotingClassifier recall :  0.867
VotingClassifier f1_score :  0.865847235864722
VotingClassifier train time :  2.8989346027374268  sec
VotingClassifier test time :  0.4742422103881836  sec


